# Testing the Gemini in Colab

this is using the REST API and runnig Curl command first

In [ ]:
import os
from google.colab import userdata

os.environ['API_KEY'] = userdata.get('Gemini_key')


In [ ]:
import json
import os
import pprint

curl = """
curl https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$API_KEY \
    -H 'Content-Type: application/json' \
    -X POST \
    -d '{
      "contents": [{
        "parts":[{
          "text": "Write a story about a magic backpack."}]}]}' 2> /dev/null
"""

# run curl comand and load response into a json dict
response = json.loads(os.popen(curl).read())

import pprint
pprint.pprint(response)

# Second Try using all python notebook

In [ ]:
pip install -q -U google-generativeai

In [ ]:
# load API key
import os
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('Gemini_key')
genai.configure(api_key=GOOGLE_API_KEY)

# load other necessary packages

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [ ]:
%%time
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("write me a short poem about mazda miata")
to_markdown(response.text)

CPU times: user 152 ms, sys: 12.3 ms, total: 164 ms
Wall time: 11.7 s


> In the realm of dreams, where legends take form,
> Lies a roadster so sleek, its beauty will charm.
> With curves that entice and a soul so refined,
> The Mazda Miata, a masterpiece of time.
> 
> Its compact frame, a symphony of grace,
> Carries a heart that beats with unbridled pace.
> A symphony of power, a dance on the road,
> As the Miata's engine sings its melodic ode.
> 
> Beneath the open sky, its top caressed by the breeze,
> It dances through corners with effortless ease.
> A nimble companion, a friend so true,
> The Miata invites you to leave the old and embrace the new.
> 
> With each turn of the key, a surge of delight,
> As the Miata's spirit takes flight.
> It whispers secrets of freedom and joy,
> As it carries you forth, a pure unadulterated toy.
> 
> So let the wind rush through your hair,
> As the Miata's symphony fills the air.
> In this realm of dreams, where legends reside,
> The Mazda Miata, a timeless ride.

In [ ]:
response.candidates


# just passing pictures to Gemini:


In [ ]:
!curl -o image.jpg https://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQ_Kevbk21QBRy-PgB4kQpS79brbmmEG7m3VOTShAn4PecDU5H5UxrJxE3Dw1JiaG17V88QIol19-3TM2wCHw

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  405k  100  405k    0     0  4990k      0 --:--:-- --:--:-- --:--:-- 5000k


In [ ]:
!curl -o image2.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Wikipedia-logo-v2-en.svg/800px-Wikipedia-logo-v2-en.svg.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  217k  100  217k    0     0   573k      0 --:--:-- --:--:-- --:--:--  573k


In [ ]:
!curl -o cat.jpg https://upload.wikimedia.org/wikipedia/commons/1/15/Cat_August_2010-4.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4523k  100 4523k    0     0  7987k      0 --:--:-- --:--:-- --:--:-- 7992k


In [ ]:
! curl -o dog.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Labrador_Retriever_portrait.jpg/1920px-Labrador_Retriever_portrait.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  407k  100  407k    0     0   747k      0 --:--:-- --:--:-- --:--:--  748k


In [ ]:
import PIL.Image

img = PIL.Image.open('image2.jpg')

In [ ]:
model = genai.GenerativeModel('gemini-pro-vision')

response = model.generate_content(img)

to_markdown(response.text)

>  The Wikipedia logo is a puzzle sphere made up of interlocking puzzle pieces. The sphere is divided into 25 pieces, each of which is a different color. The colors represent the different languages in which Wikipedia is available. The puzzle pieces are also arranged in a way that suggests that they are all connected, which represents the fact that Wikipedia is a collaborative project that is constantly being updated and improved by people from all over the world.

## text and image together

In [ ]:
img_c = PIL.Image.open('cat.jpg')
img_d = PIL.Image.open('dog.jpg')

In [ ]:
model = genai.GenerativeModel('gemini-pro-vision')
prompt='what is the difference between these images?'


response = model.generate_content([prompt, img_c,img_d], stream=True)
response.resolve()
to_markdown(response.text)

>  The first image is of a cat, and the second image is of a dog.

## Chat with gemini

In [ ]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [ ]:
response = chat.send_message("it was rainy today so no good")
to_markdown(response.text)

> **Tranquil**
> 
> Rainy weather can be tranquil and serene, providing a sense of peace and relaxation. The pitter-patter of raindrops, the smell of fresh rain, and the lush greenery that often accompanies rainy days can be soothing and calming.
> 
> Rainy weather is also a great time to curl up indoors with a good book, a warm cup of tea, and the sound of rain outside. It can be a welcome respite from the hustle and bustle of everyday life, allowing you to slow down and appreciate the simple things.
> 
> While rainy weather may not be ideal for outdoor activities, it can be a wonderful opportunity to unwind, recharge, and connect with your inner self.

In [ ]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: how many dogs should I adopt?

> **model**: The number of dogs you should adopt depends on several factors such as your lifestyle, living space, financial resources, and time commitment. Here are some considerations to help you decide:
> 
> **1. Lifestyle:**
> 
> - Do you have an active lifestyle and enjoy spending time outdoors? If so, adopting an energetic breed that requires regular exercise might be a suitable choice.
> - Do you prefer a more relaxed lifestyle and enjoy cuddles? If so, a less active breed might be a better fit.
> 
> **2. Living Space:**
> 
> - How much space do you have in your home or apartment? Some breeds require more space to roam and play, while others can adapt to smaller living quarters.
> - Consider the size of the dog when fully grown to ensure it has enough space to move comfortably.
> 
> **3. Financial Resources:**
> 
> - Owning a dog involves various expenses, including food, veterinary care, grooming, toys, and possibly pet insurance.
> - Evaluate your financial situation to determine if you can comfortably afford the costs associated with dog ownership.
> 
> **4. Time Commitment:**
> 
> - Dogs require attention, love, and training. Consider how much time you can dedicate to walks, playtime, training sessions, and vet appointments.
> - Some breeds require more mental and physical stimulation than others, so research the breed's requirements before making a decision.
> 
> **5. Family Situation:**
> 
> - If you have young children, consider adopting a breed known for being gentle and patient with kids.
> - If you have other pets, ensure the new dog will be compatible with them.
> 
> **6. Experience and Knowledge:**
> 
> - If you're a first-time dog owner, opt for a breed known for being easy to train and suitable for novice owners.
> - Consider attending obedience classes or hiring a professional dog trainer to help you bond with your new pet and establish good behaviors.
> 
> **7. Breed Research:**
> 
> - Research different dog breeds to understand their unique characteristics, temperaments, and exercise requirements.
> - Choose a breed that aligns with your lifestyle, living space, and energy level.
> 
> **8. Adoption from Shelters or Rescues:**
> 
> - Consider adopting a dog from a local animal shelter or rescue organization. These organizations often have dogs of various breeds, ages, and temperaments, and they can provide valuable information about their personalities and needs.
> 
> **9. Start with One Dog:**
> 
> - If you're a first-time dog owner, starting with one dog is advisable. This allows you to focus on building a strong bond with your pet and learning the ropes of dog ownership.
> 
> Remember that adopting a dog is a long-term commitment. Before making a decision, carefully consider all the factors involved and ensure you're prepared to provide your new furry friend with a loving and responsible home for its entire life.

> **user**: in one word which weather type is better?

> **model**: **Pleasant**
> 
> Pleasant weather is generally mild, comfortable, and enjoyable to be in. It is characterized by moderate temperatures, low humidity, and clear or partly cloudy skies. Pleasant weather is often associated with sunshine, gentle breezes, and the absence of extreme weather events such as storms, heat waves, or cold snaps.
> 
> Pleasant weather is suitable for a wide range of outdoor activities, such as walking, hiking, biking, gardening, and picnicking. It is also conducive to relaxation and spending time outdoors without feeling too hot, cold, or uncomfortable.

> **user**: it was rainy today so no good

> **model**: **Tranquil**
> 
> Rainy weather can be tranquil and serene, providing a sense of peace and relaxation. The pitter-patter of raindrops, the smell of fresh rain, and the lush greenery that often accompanies rainy days can be soothing and calming.
> 
> Rainy weather is also a great time to curl up indoors with a good book, a warm cup of tea, and the sound of rain outside. It can be a welcome respite from the hustle and bustle of everyday life, allowing you to slow down and appreciate the simple things.
> 
> While rainy weather may not be ideal for outdoor activities, it can be a wonderful opportunity to unwind, recharge, and connect with your inner self.